In [2]:
import numpy as np
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
from os.path import join

In [3]:
raw_dir = "raw/ayhan_2021/"

In [6]:
adata = sc.read_text(join(raw_dir, "exprMatrix.tsv.gz"))
meta = pd.read_csv(join(raw_dir, "meta.tsv"), sep="\t")
adata.var = meta
adata = adata.T
adata.obs = adata.obs.set_index("Cell")

umap = pd.read_csv(join(raw_dir, "Seurat_umap.coords.tsv.gz"), sep='\t', header=None, index_col=0, names=["Cell", "Seurat_umap1", "Seurat_umap2"])
assert (adata.obs.index == umap.index).all()
adata.obs[["Seurat_umap1", "Seurat_umap2"]] = umap[["Seurat_umap1", "Seurat_umap2"]]
sc.pp.filter_cells(adata, min_genes=0)
adata.write_h5ad("processed/ayhan_2021.raw.h5ad")

/n/holystore01/LABS/price_lab/Users/khou/miniconda3/lib/python3.8/site-packages/anndata/_core/anndata.py:119: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
... storing 'orig.ident' as categorical
... storing 'group' as categorical
... storing 'sex' as categorical
... storing 'batch' as categorical
... storing 'version' as categorical
... storing 'donor' as categorical
... storing 'Cluster' as categorical


In [3]:
adata = sc.read_h5ad("processed/ayhan_2021.raw.h5ad")

In [12]:
adata.obsm["X_umap"] = adata.obs[["Seurat_umap1", "Seurat_umap2"]]

In [31]:
adata

AnnData object with n_obs × n_vars = 129908 × 17180
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'group', 'age', 'sex', 'epilepsy_duration', 'batch', 'version', 'donor', 'seurat_clusters', 'epilepsy_frequency', 'Cluster', 'Seurat_umap1', 'Seurat_umap2', 'n_genes'
    uns: 'Cluster_colors'
    obsm: 'X_umap'

In [17]:
sc.set_figure_params(dpi=200)

In [ ]:
?sc.pl.umap

In [ ]:
sc.pl.umap(adata, color=["Cluster", "SATB2", "TYRO3", "PFKP", "FN1"], ncols=1, use_raw=False)

In [8]:
df_cov = pd.get_dummies(adata.obs[["n_genes", "nCount_RNA", "percent.mt", "age", "batch", "sex", "version", "epilepsy_duration", "epilepsy_frequency"]], drop_first=True)
print(np.linalg.matrix_rank(df_cov.values))

# make covariates (remove dependent columns)
df_cov = pd.get_dummies(adata.obs[["n_genes", "nCount_RNA", "percent.mt", "batch", "sex", "epilepsy_duration"]], drop_first=True)
print(np.linalg.matrix_rank(df_cov.values))
df_cov["const"] = 1.
df_cov.to_csv("processed/ayhan_2021.cov.tsv", sep='\t')

8
8


In [10]:
# processed = raw.copy()
# sc.pp.highly_variable_genes(processed)
# processed = processed[:, processed.var.highly_variable]
# sc.pp.scale(processed, max_value=10)
# sc.tl.pca(processed, svd_solver='arpack')
# sc.pp.neighbors(processed, n_neighbors=10, n_pcs=40)
# sc.tl.umap(processed)
# raw.obsm["X_umap"] = raw.obs[['Seurat_umap1', 'Seurat_umap2']].values
# # visualize raw dataset
# sc.pl.umap(raw, color=["Cluster", "age", "sex", "batch"])
# for g in np.unique(raw.obs["orig.ident"]):
#     sub_adata = raw[raw.obs["orig.ident"] == g]
#     sc.pl.umap(sub_adata, color="Cluster")

# # visualize processed dataset
# for g in np.unique(processed.obs["orig.ident"]):
#     sub_adata = processed[processed.obs["orig.ident"] == g]
#     sc.pl.umap(sub_adata, color="Cluster")